In [ ]:

## Libraries Prequisities
!pip install scikit-learn
!pip install numpy==1.16.1

In [ ]:
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
from tensorflow.keras.callbacks import Callback

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/")
!ls
import os
os.chdir("My Drive/Computer_Vision_Masters/Wearable_Sensors_Code/")

In [ ]:
np.random.seed(12227)

In [ ]:
def DataPreparation(data_input_file):
  print('David M Burns et al. 2018 {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
 
  return X,Y,folds

In [ ]:
x,y,f=DataPreparation('/content/drive/My Drive/Computer_vision_project/data/LOSO/WHARF.npz')

In [ ]:
def ReportAccuracies(avg_acc, avg_recall,avg_f1):
  ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
  ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
  ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
  print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
  print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
  print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))

In [ ]:
def Run_David_Burns(data_input_file):
  X,Y,folds,= DataPreparation(data_input_file)
 
  avg_acc, avg_recall,avg_f1= Train(X,Y,folds)
  ReportAccuracies(avg_acc, avg_recall,avg_f1 )

In [ ]:
from tensorflow.keras.layers import Lambda

In [ ]:
def fcn(input_shape, num_classes, conv_layers=({'f': 128, 'k': 8, 's': 1}, {'f': 256, 'k': 5, 's': 1}, {'f': 128, 'k': 3, 's': 1}),
        dropout=0.3, normalize=True, embedding_size=None):
    """
    Creates fully convolutional neural (FCN) network architecture described in: https://arxiv.org/abs/2001.05517
    :param input_shape: tuple (2)
        segment shape (width, n_channels)
    :param conv_layers: tuple of dicts
        describe conv layers with f: filters, k: kernel size, s: stride
    :param dropout: float
        dropout ratio applied at each layer
    :param normalize: bool
        apply l2 normalization
    :param embedding_size: integer, optional
        defines embedding size (number of filters for last CNN layer)
    :return: keras model
        the fcn model
    """

    input_layer = tf.keras.Input(shape=input_shape)
    layer = input_layer

    if embedding_size:
        conv_layers[-1]['f'] = embedding_size

    for lp in conv_layers:
        layer = tf.keras.layers.Conv1D(filters=lp['f'], kernel_size=lp['k'], strides=lp['s'], padding='same')(layer)
        layer = tf.keras.layers.BatchNormalization()(layer)
        layer = tf.keras.layers.Activation('relu')(layer)

        if dropout:
            layer = tf.keras.layers.Dropout(rate=dropout)(layer)

    layer = tf.keras.layers.GlobalAveragePooling1D()(layer)

    if normalize:
        layer = Lambda(lambda x: tf.keras.backend.l2_normalize(x, axis=1))(layer)
    layer= tf.keras.layers.Dense(num_classes, activation = 'softmax')   (layer) 

    return tf.keras.Model(inputs=input_layer, outputs=layer)

In [ ]:
def Train(X,y,folds):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  n_class = y.shape[1]
  _,img_rows, img_cols = X.shape
  for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]
    X_train = X[train_idx]
    X_test = X[test_idx]
    model=fcn((img_rows,img_cols),n_class)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='SGD')
    model.fit(X_train, y[train_idx], batch_size=64, epochs=150)
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y[test_idx], axis=1)
    acc_fold = accuracy_score(y_true, y_pred)
    avg_acc.append(acc_fold)
    recall_fold = recall_score(y_true, y_pred, average='macro')
    avg_recall.append(recall_fold)
    f1_fold = f1_score(y_true, y_pred, average='macro')
    avg_f1.append(f1_fold)
    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
    print('________________________________________________________________')
    
  return avg_acc, avg_recall,avg_f1

In [ ]:
tf.keras.backend.set_image_data_format('channels_first')

In [ ]:
Run_David_Burns('/content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz')

In [ ]:
Run_David_Burns('/content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD1_1s.npz')

In [ ]:
Run_David_Burns('/content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD2_1s.npz')

In [ ]:
Run_David_Burns('/content/drive/My Drive/Computer_vision_project/data/LOSO/WHARF.npz')

In [ ]:
Run_David_Burns('data/LOSO/WISDM.npz')

In [ ]:
Run_David_Burns('data/FNOW/MHEALTH.npz')

In [ ]:
Run_David_Burns('data/FNOW/UTD-MHAD1_1s.npz')

In [ ]:
Run_David_Burns('data/FNOW/UTD-MHAD2_1s.npz')

In [ ]:
Run_David_Burns('data/FNOW/WHARF.npz')

In [ ]:
Run_David_Burns('data/FNOW/WISDM.npz')

In [ ]:
Run_David_Burns('data/LOTO/MHEALTH.npz')

In [ ]:
Run_David_Burns('data/LOTO/UTD-MHAD1_1s.npz')

In [ ]:
Run_David_Burns('data/LOTO/UTD-MHAD2_1s.npz')

In [ ]:
Run_David_Burns('data/LOTO/WHARF.npz')

In [ ]:
Run_David_Burns('data/SNOW/MHEALTH.npz')

In [ ]:
Run_David_Burns('data/SNOW/UTD-MHAD1_1s.npz')

In [ ]:
Run_David_Burns('data/SNOW/UTD-MHAD2_1s.npz')

In [ ]:
Run_David_Burns('data/SNOW/WHARF.npz')

In [ ]:
Run_David_Burns('data/SNOW/WISDM.npz')